In [1]:
#@title ## Script de Comparação Estatística para Métricas de Avaliação de Modelos (Upload em Duas Etapas)

#@markdown ### Instruções:
#@markdown 1. Execute esta célula.
#@markdown 2. **Passo 1: Upload do JSON do Modelo Base.**
#@markdown    - Clique no botão "Choose Files" (ou "Escolher arquivos") que aparecer e faça o upload do seu **único arquivo JSON do modelo base**.
#@markdown 3. **Passo 2: Upload dos JSONs dos Modelos de Comparação.**
#@markdown    - Após o upload do modelo base, outro botão "Choose Files" aparecerá.
#@markdown    - Faça o upload dos **quatro outros arquivos JSON dos modelos de comparação**.
#@markdown 4. O script realizará testes t pareados, calculará o d de Cohen e os ICs de 95% para a diferença média de cada métrica, comparando cada um dos quatro modelos com o modelo base. Os resultados serão compilados em um arquivo CSV.

# bibliotecas necessárias
import json
from google.colab import files
import pandas as pd
from scipy import stats
import numpy as np
import csv
# math não é explicitamente necessário pois np.sqrt é usado e stats.sem também.

In [ ]:
def calcular_estatisticas(dados_modelo_comparacao, dados_modelo_base, nome_metrica, alfa=0.05):
    """
    Calcula teste t pareado, d de Cohen e intervalo de confiança para amostras pareadas.

    Args = 
        dados_modelo_comparacao (pd.Series): Scores da métrica para o modelo de comparação.
        dados_modelo_base (pd.Series): Scores da métrica para o modelo base.
        nome_metrica (string): Nome da métrica sendo analisada.
        alfa (float): Nível de significância para o intervalo de confiança (padrão 0.05 para IC de 95%).

    Returns =
        dict: Um dicionário contendo todas as estatísticas calculadas.
              Retorna None se os dados forem insuficientes
    """
    if len(dados_modelo_comparacao) < 2 or len(dados_modelo_base) < 2:
        print(f"Aviso: Dados insuficientes para a métrica '{nome_metrica}'. São necessárias pelo menos 2 amostras pareadas.")
        return None
    if len(dados_modelo_comparacao) != len(dados_modelo_base):
        # Isso idealmente não deveria acontecer se o alinhamento e a limpeza de NaN (Not A Number) forem feitos antes. Mas é bom prevenir de qualquer forma.
        print(f"Aviso: Tamanhos de amostra incompatíveis para a métrica '{nome_metrica}' ({len(dados_modelo_comparacao)} vs {len(dados_modelo_base)}). Pulando.")
        return None

    # Teste t pareado
    # Nota: stats.ttest_rel já lida com NaNs se nan_policy='omit' for usado, mas já foi limpo os NaNs nas etapas anteriores para garantir o pareamento correto.
    estatistica_t, valor_p = stats.ttest_rel(dados_modelo_comparacao, dados_modelo_base, nan_policy='omit')

    diferencas = dados_modelo_comparacao - dados_modelo_base
    n_pares = len(diferencas.dropna()) # Número de pares não-NaN

    if n_pares < 2:
        print(f"Aviso: Dados pareados não-NaN insuficientes para a métrica '{nome_metrica}' após calcular as diferenças. São necessários pelo menos 2 pares.")
        return None

    media_diferencas = np.mean(diferencas)
    desvio_padrao_diferencas = np.std(diferencas, ddof=1) # ddof=1 para desvio padrão amostral

    if desvio_padrao_diferencas == 0:
        cohen_d = float('inf') if media_diferencas != 0 else 0.0
    else:
        cohen_d = media_diferencas / desvio_padrao_diferencas

    gl = n_pares - 1 # Graus de liberdade
    erro_padrao_media_diferencas = stats.sem(diferencas, nan_policy='omit')

    if erro_padrao_media_diferencas == 0 and gl > 0:
        ic_inferior, ic_superior = media_diferencas, media_diferencas
    elif gl <= 0 :
        ic_inferior, ic_superior = float('nan'), float('nan')
    else:
        ic_inferior, ic_superior = stats.t.interval(1 - alfa, gl, loc=media_diferencas, scale=erro_padrao_media_diferencas)

    if abs(cohen_d) < 0.01:
        interpretacao_cohen_d = "Efeito insignificante"
    elif abs(cohen_d) < 0.2:
        interpretacao_cohen_d = "Efeito muito pequeno"
    elif abs(cohen_d) < 0.5:
        interpretacao_cohen_d = "Efeito pequeno"
    elif abs(cohen_d) < 0.8:
        interpretacao_cohen_d = "Efeito médio"
    else:
        interpretacao_cohen_d = "Efeito grande"

    return {
        "Metrica": nome_metrica,
        "N_Pares": n_pares,
        "Media_Diferencas": media_diferencas,
        "Desvio_Padrao_Diferencas": desvio_padrao_diferencas,
        "Estatistica_T": estatistica_t,
        "GL": gl,
        "Valor_P": valor_p,
        "Cohen_D": cohen_d,
        "Interpretacao_Cohen_D": interpretacao_cohen_d,
        f"IC_{(1-alfa)*100:.0f}%_Inferior": ic_inferior,
        f"IC_{(1-alfa)*100:.0f}%_Superior": ic_superior,
        "Significativo_Alpha_005": valor_p < 0.05
    }

In [ ]:
def carregar_e_preparar_df(conteudo_arquivo, nome_arquivo):
    """Carrega o conteúdo JSON em um dataframe pandas"""
    try:
        dados = json.loads(conteudo_arquivo.decode('utf-8'))
        df = pd.DataFrame(dados)
        if 'doc_id' not in df.columns:
            print(f"Erro: Arquivo {nome_arquivo} não possui a coluna 'doc_id'. Não é possível prosseguir.")
            return None, None
        df = df.set_index('doc_id').sort_index() # Define doc_id como índice para fácil alinhamento dos dados
        metricas = [col for col in df.columns if col not in ['doc_id']] # Assume que as outras colunas são métricas
        return df, metricas
    except json.JSONDecodeError:
        print(f"Erro: Não foi possível decodificar o JSON do arquivo {nome_arquivo}. Por favor, garanta que é um JSON válido.")
        return None, None
    except Exception as e:
        print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")
        return None, None

In [4]:
lista_todos_resultados = []

print("--- Passo 1: Upload do Modelo Base ---")
print("Por favor, faça o upload do ÚNICO arquivo JSON do modelo base.")
arquivos_base_carregados = files.upload()

--- Passo 1: Upload do Modelo Base ---
Por favor, faça o upload do ÚNICO arquivo JSON do modelo base.


Saving llama2base.json to llama2base.json


In [ ]:
#@markdown ---
#@markdown ### ⚠️ Nota para Abrir o CSV Corretamente:
#@markdown Se o arquivo CSV parecer desorganizado ao abrir no Excel (ou similar):
#@markdown 1. Tente usar a opção **`Dados`** -> **`De Texto/CSV`** (ou **`Importar Dados de Texto`**).
#@markdown 2. Na janela de importação, verifique de que o **`Delimitador`** está configurado para **`Vírgula (,)`**.
#@markdown 3. Verifique se a **`Origem do Arquivo`** ou **`Codificação`** está configurada para **`65001: Unicode (UTF-8)`**.


if len(arquivos_base_carregados) != 1:
    print(f"Erro: Esperado 1 arquivo de modelo base, mas {len(arquivos_base_carregados)} foram recebidos. Por favor, execute novamente e faça o upload de exatamente 1 arquivo para o modelo base.")
else:
    nome_modelo_base = list(arquivos_base_carregados.keys())[0]
    conteudo_modelo_base = arquivos_base_carregados[nome_modelo_base]
    df_base, metricas_para_analisar = carregar_e_preparar_df(conteudo_modelo_base, nome_modelo_base)

    if df_base is None:
        print("Falha ao carregar o modelo base. Saindo.")
    else:
        print(f"\nModelo base '{nome_modelo_base}' carregado com sucesso.")
        if not metricas_para_analisar:
            print("Nenhuma coluna de métrica encontrada no modelo base (além de 'doc_id'). Verifique o arquivo JSON.")
        else:
            print(f"Métricas a serem analisadas (derivadas do modelo base): {', '.join(metricas_para_analisar)}")

            print("\n--- Passo 2: Upload dos Modelos de Comparação ---")
            print("Por favor, faça o upload dos QUATRO arquivos JSON dos modelos de comparação.")
            arquivos_comparacao_carregados = files.upload()

            if len(arquivos_comparacao_carregados) != 4:
                print(f"Erro: Esperados 4 arquivos de modelos de comparação, mas {len(arquivos_comparacao_carregados)} foram recebidos. Por favor, execute novamente.")
            else:
                dados_modelos_comparacao = []
                arquivos_comparacao_validos = True
                for nome_comparacao, conteudo_comparacao in arquivos_comparacao_carregados.items():
                    df_comparacao, _ = carregar_e_preparar_df(conteudo_comparacao, nome_comparacao) # Métricas do teste não são primariamente usadas aqui para definir a lista de métricas
                    if df_comparacao is None:
                        arquivos_comparacao_validos = False
                        break
                    dados_modelos_comparacao.append({'nome': nome_comparacao, 'df': df_comparacao})

                if not arquivos_comparacao_validos:
                    print("Falha ao carregar um ou mais modelos de comparação. Saindo.")
                else:
                    print("\nTodos os modelos de comparação carregados com sucesso.")
                    print("\n--- Processando Comparações Estatísticas ---")

                    for info_modelo_comparacao in dados_modelos_comparacao:
                        nome_modelo_comparacao = info_modelo_comparacao['nome']
                        df_comparacao = info_modelo_comparacao['df']

                        print(f"\nComparando MODELO DE COMPARAÇÃO: '{nome_modelo_comparacao}' vs MODELO BASE: '{nome_modelo_base}'")

                        # Alinhar DataFrames por doc_id (índice)
                        df_base_alinhado, df_comparacao_alinhado = df_base.align(df_comparacao, join='inner', axis=0)

                        if df_base_alinhado.empty or df_comparacao_alinhado.empty:
                            print(f"  Nenhum 'doc_id' comum encontrado entre '{nome_modelo_comparacao}' e '{nome_modelo_base}'. Pulando comparação.")
                            continue

                        print(f"  Número de documentos comuns para comparação: {len(df_base_alinhado)}")

                        for metrica in metricas_para_analisar:

                            if metrica not in df_base_alinhado.columns:
                                print(f"   Aviso: Métrica '{metrica}' não encontrada nos dados alinhados do modelo base. Pulando.")
                                continue
                            if metrica not in df_comparacao_alinhado.columns:
                                print(f"   Aviso: Métrica '{metrica}' não encontrada nos dados alinhados do modelo de comparação ('{nome_modelo_comparacao}'). Pulando.")
                                continue

                            dados_metrica_base = df_base_alinhado[metrica]
                            dados_metrica_comparacao = df_comparacao_alinhado[metrica]

                            # Remover pares onde qualquer valor é NaN APÓS o alinhamento para a métrica específica
                            indices_validos = ~(dados_metrica_base.isna() | dados_metrica_comparacao.isna())
                            dados_metrica_base_limpos = dados_metrica_base[indices_validos]
                            dados_metrica_comparacao_limpos = dados_metrica_comparacao[indices_validos]

                            if len(dados_metrica_base_limpos) < 2: # Precisa de pelo menos 2 pares para o teste t
                                print(f"  Métrica '{metrica}': Não há pontos de dados pareados válidos (não-NaN) suficientes para análise estatística.")
                                continue

                            resultados_estatisticos = calcular_estatisticas(dados_metrica_comparacao_limpos, dados_metrica_base_limpos, metrica)

                            if resultados_estatisticos:
                                # Adicionar nomes dos modelos aos resultados para o CSV
                                resultados_completos = {
                                    "Nome_Modelo_Base": nome_modelo_base,
                                    "Nome_Modelo_Comparacao": nome_modelo_comparacao,
                                    **resultados_estatisticos # "Desempacota" o dicionário de estatísticas
                                }
                                lista_todos_resultados.append(resultados_completos)
                                print(f"  Métrica '{metrica}': Análise concluída. {resultados_estatisticos['N_Pares']} pares.")

                                # Interpretação básica (opcional para console, pois estará no CSV, mas foi utilizada para ajudar no debug)
                                if resultados_estatisticos['Significativo_Alpha_005']:
                                    comp = "MAIOR" if resultados_estatisticos['Media_Diferencas'] > 0 else "MENOR" if resultados_estatisticos['Media_Diferencas'] < 0 else "IGUAL (mas p<0.05)"
                                    print(f"    Resultado para '{metrica}': Estatisticamente significativo. Modelo de comparação é significativamente {comp}.")
                                else:
                                    print(f"    Resultado para '{metrica}': Não estatisticamente significativo.")

                            else:
                                print(f"  Métrica '{metrica}': Não foi possível calcular as estatísticas.")

                    # Após todos os loops, criar e baixar o CSV
                    if lista_todos_resultados:
                        df_final_resultados = pd.DataFrame(lista_todos_resultados)

                        colunas_ordenadas = [
                            "Nome_Modelo_Base", "Nome_Modelo_Comparacao", "Metrica", "N_Pares",
                            "Media_Diferencas", "Desvio_Padrao_Diferencas", "Estatistica_T", "GL", "Valor_P",
                            "Cohen_D", "Interpretacao_Cohen_D",
                            f"IC_95%_Inferior", f"IC_95%_Superior",
                            "Significativo_Alpha_005"
                        ]
                        colunas_existentes_ordenadas = [col for col in colunas_ordenadas if col in df_final_resultados.columns]

                        df_final_resultados = df_final_resultados[colunas_existentes_ordenadas]

                        nome_arquivo_csv = 'resultados_comparacao_modelos.csv'
                        df_final_resultados.to_csv(nome_arquivo_csv, index=False, encoding='utf-8-sig', sep=',',quoting=csv.QUOTE_NONNUMERIC ) # utf-8-sig para melhor compatibilidade com Excel
                        print(f"\n\n--- Análise Concluída ---")
                        print(f"Os resultados foram salvos em '{nome_arquivo_csv}'. Iniciando download...")
                        files.download(nome_arquivo_csv)
                    else:
                        print("\n\n--- Análise Concluída ---")
                        print("Nenhum resultado estatístico foi gerado para criar um arquivo CSV.")

print("\n--- Fim do Script ---")


Modelo base 'llama2base.json' carregado com sucesso.
Métricas a serem analisadas (derivadas do modelo base): rouge1, rouge2, rougeL, rougeLsum, bertscore_f1, bertscore_precision, bertscore_recall

--- Passo 2: Upload dos Modelos de Comparação ---
Por favor, faça o upload dos QUATRO arquivos JSON dos modelos de comparação.


Saving icd.json to icd.json
Saving kca.json to kca.json
Saving repe.json to repe.json
Saving wikichat.json to wikichat.json

Todos os modelos de comparação carregados com sucesso.

--- Processando Comparações Estatísticas ---

Comparando MODELO DE COMPARAÇÃO: 'icd.json' vs MODELO BASE: 'llama2base.json'
  Número de documentos comuns para comparação: 1000
  Métrica 'rouge1': Análise concluída. 1000 pares.
    Resultado para 'rouge1': Estatisticamente significativo. Modelo de comparação é significativamente MAIOR.
  Métrica 'rouge2': Análise concluída. 1000 pares.
    Resultado para 'rouge2': Estatisticamente significativo. Modelo de comparação é significativamente MAIOR.
  Métrica 'rougeL': Análise concluída. 1000 pares.
    Resultado para 'rougeL': Estatisticamente significativo. Modelo de comparação é significativamente MAIOR.
  Métrica 'rougeLsum': Análise concluída. 1000 pares.
    Resultado para 'rougeLsum': Estatisticamente significativo. Modelo de comparação é significativamente 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- Fim do Script ---
